In [1]:
import numpy as np
import pandas as pd
import random as rnd
from random import randint
import panel as pn
import panel.widgets as pnw
import matplotlib.pyplot as plt
from time import time
from numpy.linalg import norm
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvas
from matplotlib.patches import Polygon as plt_Polygon
from matplotlib.collections import PatchCollection

import psycopg2
import math
from matplotlib.animation import FuncAnimation
import datetime
import csv
from postgis import Polygon,MultiPolygon
from postgis.psycopg import register

from time import time
import time
import progressbar

# 
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import shapely.geometry as sg
import shapely.ops as so

In [2]:
data = pd.read_csv('../../csv_s/out.csv')

def tranfFunc(x):
    y = x[1: -1].split(',')
    return [float(y[0]), float(y[1])]

src = data['Src'].transform(tranfFunc).to_numpy()
dst = data['Dst'].transform(tranfFunc).to_numpy()


In [3]:
conn = psycopg2.connect(database="client-project-places", user="postgres", password='', host="127.0.0.1", port="9876")
print(conn)

<connection object at 0x7fe45a434e10; dsn: 'user=postgres password=xxx dbname=client-project-places host=127.0.0.1 port=9876', closed: 0>


In [4]:
places_matrix = []

register(conn)
cursor_psql = conn.cursor()

for x in progressbar.progressbar(dst[0:10]):
    district = council = parish = '-'
    sql="""
    SELECT distrito, concelho, freguesia 
        FROM parishes 
        WHERE ST_Contains(
            geom, ST_GeometryFromText(
                ST_AsText(
                    ST_Transform(
                        ST_GeomFromText('POINT( """+ str(x[0]) + """ """+ str(x[1]) + """)',4326),3763
                    )
                ),3763
            )
        )
    """
    cursor_psql.execute(sql)
    results = cursor_psql.fetchall()

    for row in results:
        district, council, parish = row[0], row[1], row[2]

    places_matrix.append([district, council, parish])
                
df = pd.DataFrame(places_matrix)

100% (10 of 10) |########################| Elapsed Time: 0:00:01 Time:  0:00:01


In [46]:
# df.to_csv('dst.csv')

In [5]:
df

,0,1,2
0,PORTO,PORTO,União das freguesias de Lordelo do Ouro e Mass...
1,PORTO,PORTO,União das freguesias de Lordelo do Ouro e Mass...
2,PORTO,PORTO,Campanhã
3,PORTO,PORTO,"União das freguesias de Cedofeita, Santo Ildef..."
4,PORTO,PORTO,União das freguesias de Lordelo do Ouro e Mass...
5,PORTO,MATOSINHOS,União das freguesias de São Mamede de Infesta ...
6,PORTO,PORTO,Ramalde
7,PORTO,VILA NOVA DE GAIA,Canidelo
8,PORTO,PORTO,Paranhos
9,PORTO,VILA NOVA DE GAIA,Oliveira do Douro
